# Image Processing SS 20 - Assignment - 09

### Deadline is 24.6.2020 at 11:55am

Please solve the assignments together with a partner.
I will run every notebook. Make sure the code runs through. Select `Kernel` -> `Restart & Run All` to test it.
Please strip the output from the cells, either select `Cell` -> `All Output` -> `Clear` or use the `nb_strip_output.py` script / git hook.

In [ ]:
# display the plots inside the notebook
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab
from scipy.fftpack import dct, idct
from skimage.data import chelsea
import zipfile
pylab.rcParams['figure.figsize'] = (15, 15)   # This makes the plot bigger

# JPEG

The Wikipedia page about [JPEG-Komprimierung](https://de.wikipedia.org/wiki/JPEG#Die_JPEG-Komprimierung) gives
a good overview. Wikipedia assume that the image data is uint8. That is why we do not convert it to float in this assignment.

In [ ]:
# Let's start with the input image
img = chelsea()

block_size = 8
# add some padding so the image can be evenly sliced into blocks
pad_x = 0 if img.shape[0] % (block_size * 2) == 0 else (img.shape[0] // (block_size * 2) + 1) * (block_size * 2) - img.shape[0]
pad_y = 0 if img.shape[1] % (block_size * 2) == 0 else (img.shape[1] // (block_size * 2) + 1) * (block_size * 2) - img.shape[1]
img = np.pad(img, ((0, pad_x), (0, pad_y), (0, 0)))

In [ ]:
class Blocks:
    """Transforms an image to blocks. A (512, 512) image will become an (64, 64, 8, 8) numpy array"""
    def __init__(self, block_size=8):
        self.block_size = block_size
        
    def __call__(self, img):
        b = self.block_size
        height, width = img.shape

        assert img.shape[0] % b == 0
        assert img.shape[1] % b == 0
        blocks = np.zeros((height // b, width // b, b, b), dtype=img.dtype)
        for i in range(0, height // b):
            for j in range(0, width // b):
                blocks[i, j] = img[i*b:(i+1)*b, j*b:(j+1)*b]
        return blocks

    def invert(self, blocks):
        bh, bw = blocks.shape[:2]
        b = self.block_size
        heigth, width = (bh*self.block_size, bw*self.block_size)

        img = np.zeros((heigth, width), dtype=blocks.dtype)
        for i in range(0, bh):
            for j in range(0, bw):
                img[i*b:(i+1)*b, j*b:(j+1)*b] = blocks[i, j]
        return img

# test block operation on R-Channel
img_blocks = Blocks(block_size=8)(img[:, :, 0])
print(img_blocks.shape)
assert img_blocks.shape[2:] == (8, 8)
img_inv = Blocks(block_size=8).invert(img_blocks)

# the two images should be the same again after inverting
assert (img[:, :, 0] == img_inv).all()

plt.subplot(121)
plt.imshow(img[:, :, 0], cmap='gray')
plt.subplot(122)
plt.imshow(img_inv, cmap='gray')
plt.show()

# Entropy encoding
This class applies the zigzag algorithm to compress the matrix

In [ ]:
class ZigZag:
    def __init__(self, n):
        self.n = n
        """Adapted from here: https://rosettacode.org/wiki/Zig-zag_matrix#Python"""
        def key(pair):
            x, y = pair
            return x+y, -y if (x+y) % 2 else y

        n = 8
        indexorder = sorted(((x,y) for x in range(n) for y in range(n)), key=key)
        self.xs = np.zeros((self.n**2,), dtype=np.int)
        self.ys = np.zeros((self.n**2,), dtype=np.int)
        self.back = np.zeros((n, n), dtype=np.int)
        for i, (x, y) in enumerate(indexorder):
            self.xs[i] = x
            self.ys[i] = y
            self.back[x, y] = i
            
    def __call__(self, blocks):
        bh, bw, h, w = blocks.shape
        zigzack_blocks = np.zeros((bh, bw, h*w), dtype=blocks.dtype)
        for i, block_row in enumerate(blocks):
            for j, block in enumerate(block_row):
                zigzack_blocks[i, j] = block[self.xs, self.ys] 
        return zigzack_blocks
    
    def invert(self, zigzack_blocks):
        bh, bw, hw = zigzack_blocks.shape
        h = int(np.sqrt(hw))
        blocks = np.zeros((bh, bw, h, h), dtype=zigzack_blocks.dtype)
        for i, zigzack_row in enumerate(zigzack_blocks):
            for j, zigzack in enumerate(zigzack_row):
                blocks[i, j] = zigzack[self.back] 
        return blocks

zigzag = ZigZag(8)
range_mat = np.arange(64).reshape(1, 1, 8, 8)
zigzack_mat = zigzag(range_mat)
assert (zigzag.invert(zigzack_mat) == range_mat).all()

print(range_mat)
print(zigzack_mat)

In [ ]:
class Jpeg:
    def __init__(self, stages):
        self.stages = stages
    
    def __call__(self, img):
        y, cb, cr = ChromaSubsampling()(img)
        outputs = []
        for input in [y, cb, cr]:
            output = input
            for stage in self.stages:
                input = output
                try:
                    output = stage(input)
                except:
                    print("Error in Stage: {}".format(type(stage).__name__))
                    raise
            outputs.append(output)
        return outputs
    
    def invert(self, inputs):
        outputs = []
        for output in inputs:
            for stage in self.stages[::-1]:
                input = output
                try:
                    output = stage.invert(input)
                except:
                    print("Error in Stage: {}".format(type(stage).__name__))
                    raise
            outputs.append(output)
        y, cb, cr = outputs
        return ChromaSubsampling().invert([y, cb, cr])

In [ ]:
class Compress:
    """This class compresses arbitray input using the bz2 algorithm"""
    def __init__(self, dtype=np.int8):
        self.dtype = dtype
        self.max_value = (np.iinfo(dtype).max  / 1.1)
        
    def __call__(self, arr):
        # print("dtype: {}".format(arr.dtype))
        # print("max: {}, min: {}".format(arr.max(), arr.min()))
        scale = max(abs(arr.max()), abs(arr.min()))
        arr = arr / scale 
        arr = arr * self.max_value
        arr = np.rint(arr)
        arr = arr.astype(self.dtype)
        bytearr = arr.data.tobytes()
        
        return zipfile.bz2.compress(bytearr), arr.shape, arr.dtype, scale
    
    def invert(self, inputs):
        bytearr, shape, dtype, scale = inputs
        decom_bytes = zipfile.bz2.decompress(bytearr)
        arr = np.frombuffer(decom_bytes, dtype=self.dtype)
        arr = arr.astype(dtype)
        arr = arr / self.max_value * scale
        return arr.reshape(shape)

range_mat = np.arange(64, dtype=np.float64).reshape(1, 1, 8, 8)
compress = Compress()
compressed = compress(range_mat)
# print((compress.invert(compressed)))
assert np.allclose(compress.invert(compressed), range_mat, 0.5, 0.5)

In [ ]:
def total_size_jpeg(jpeg_output):
    """Summs the number of bytes over the different compression channels: y, cb, cr"""
    nb_bytes = sum([len(x[0]) for x in jpeg_output])
    return (nb_bytes / 1000)

def total_size_numpy(arr):
    return (len(arr.data.tobytes()) / 1000)

def naive_compression_size(arr):
    bytearr = arr.data.tobytes()
    nb_bytes = len(zipfile.bz2.compress(bytearr))
    return (nb_bytes / 1000)

In [ ]:
chromasubsampling_compression = Jpeg([Blocks(8), DCTofBlocks(), Quantization(threshold=50), ZigZag(8), Compress(np.int8)])
print("No compression: " + str(total_size_numpy(img)) )
print("Direct compression of the image: " + str(naive_compression_size(img)))
print("Cromasubsampling and compression: " + str(total_size_jpeg(chromasubsampling_compression(img))))

# Exercise 1 - Chroma Subsampling - 2 Points
Apply chroma subsampling to the image.

In [ ]:
def ycbcr2rgb(im):
    xform = np.array([[1, 0, 1.402], [1, -0.34414, -.71414], [1, 1.772, 0]])
    rgb = im.astype(np.float)
    rgb[:,:,[1,2]] -= 128
    rgb = rgb.dot(xform.T)
    np.putmask(rgb, rgb > 255, 255)
    np.putmask(rgb, rgb < 0, 0)
    return np.uint8(rgb)

In [ ]:
class ChromaSubsampling:
    """See https://en.wikipedia.org/wiki/YCbCr."""
    ycbcr = np.array([
        [0.299,  0.587,  0.114],
        [-0.168736, -0.331264,  0.5],
        [0.5, -0.418688, -0.0813],
    ])
    def __call__(self, rgb_img):
        """Transforms the rgb image to YCbCr. The cb and cr channels have half the resolution of the Y-channel.
           You can simply use the mean of four neighbours.
        """
        shape = rgb_img.shape
        
        ycbcr_img = np.dot(self.ycbcr, rgb_img.reshape(-1, 3).T)
        ycbcr_img = ycbcr_img.T.reshape(shape)
        ycbcr_img[:,:, [1,2]] += 128
        # subsample the cb and cr channel, so that they have half the resolution of the Y-channel.
        # A simple thing might be to use the mean of 4 neighbours.
        # use 4:2:0 
        cb = ycbcr_img[:,:,1]
        cr = ycbcr_img[:,:,2]
        for i in range(0,shape[0],2):
            for j in range(0,shape[1],2):             
                kernel_cb = cb[i:i+2,j:j+2]
                kernel_cr = cr[i:i+2,j:j+2]
                mean_cb = np.mean(kernel_cb)
                mean_cr = np.mean(kernel_cr)
                cb[i:i+2,j:j+2] = mean_cb
                cr [i:i+2,j:j+2] = mean_cr
        ycbcr_img[:,:,1] = cb
        ycbcr_img[:,:,2] = cr 
        return ycbcr_img[:, :, 0], ycbcr_img[:, :, 1], ycbcr_img[:, :, 2]
    
    def invert(self, inputs):
        y, cb, cr = inputs
        # your code here 
        shape = [y.shape[0],y.shape[1],3]
        result= np.zeros(shape)
        result[:,:,0]=y
        result[:,:,1]=cb
        result[:,:,2]=cr
        return ycbcr2rgb(result)

Test the subsampling: The cr and cb channel should be half the resolution of the y channel.

In [ ]:
y, cb, cr = ChromaSubsampling()(img)
plt.subplot(141)
plt.title('Luminance Y')
plt.imshow(y, cmap='gray')
plt.subplot(142)
plt.title('Subsampled Chrominance Cr')
plt.imshow(cb, cmap='gray')
plt.subplot(143)
plt.title('Subsampled Chrominance Cb')
plt.imshow(cr, cmap='gray')
plt.subplot(144)
plt.title('Inverted')
plt.imshow(ChromaSubsampling().invert((y,cb,cr)) ,cmap='gray')
plt.show()

# Exercise 2 - DCT on 8x8 blocks - 2 Points
Apply DCT on 8x8 blocks. You may use np.fft` or `scipy.fftpack`.

In [ ]:
class DCTofBlocks:
    def __call__(self, blocks):
        """Returns the DCT of the blocks. The position (i, j) is a 2-dim numpy array with the dct coefficents."""
        # you can use any function from np.fft or scipy.fftpack
        # your code here 
        x ,y = blocks.shape[0], blocks.shape[1]
        result = np.zeros_like(blocks)
        for i in range(x):
            for j in range(y):
                result[i,j] = dct(dct(blocks[i,j].T, norm='ortho').T, norm='ortho') # dct-II ortho
        return result.astype(int)
    
    def invert(self, blocks):
        """Computes the inverse DCT."""
        # you can use any function from np.fft or scipy.fftpack
        # your code here
        result = np.zeros_like(blocks)
        x ,y = blocks.shape[0], blocks.shape[1]
        for i in range(x):
            for j in range(y):
                result[i,j] = idct(idct(blocks[i,j].T, norm='ortho').T, norm='ortho')
        return result.astype(int)

In [ ]:
# print some blocks and their DCT on the y channel
y_block = Blocks()(y)
y_block_dct = DCTofBlocks()(y_block)

# show full image
dct_img = Blocks().invert(y_block_dct)
plt.imshow(dct_img, cmap='gray', vmax = np.max(dct_img) * 0.01,vmin = 0)
plt.show()


# apply inverse DCT
a = Blocks().invert(DCTofBlocks().invert(y_block_dct))
plt.imshow( a , cmap='gray')
plt.show()


# Exercise 3 - Quantization - 2 Points
Apply quantization on the image with a threshold.

In [ ]:
class Quantization:
    def __init__(self, threshold=2):
        # you can use the Q matrix from Wikipedia or invent your own.

        self.q_matrix = np.array([[10,15,25,37,51,66,82,100],
                          [15,19,28,39,52,67,83,101],
                          [25,28,35,45,58,72,88,105],
                          [37,39,45,54,66,79,94,111],
                          [51,52,58,66,76,89,103,119],
                          [66,67,72,79,89,101,114,130],
                          [82,83,88,94,103,114,127,142],
                          [100,101,105,111,119,130,142,156]])

        self.threshold = threshold

    def __call__(self, blocks):
        """Divides the blocks by the `q_matrix` elementwise. Coefficents under the `threshold` will be set to zero."""
        result = blocks.copy()
        x,y = blocks.shape[0],blocks.shape[1]
        for i in range(x):
            for j in range(y):
                tmp = result[i,j] // self.q_matrix
                np.putmask(tmp,tmp < self.threshold, 0)
                result[i,j] = tmp 
        return result
    
    def invert(self, blocks):
        """ For inverting multiply your elements piecewise with the Q-Matrix"""
        result = blocks.copy()
        x,y = blocks.shape[0],blocks.shape[1]
        for i in range(x):
            for j in range(y):
                tmp = result[i,j] * self.q_matrix
                result[i,j] = tmp 
        return result

Try out Quantization. You should see that with increasing threshold the image becomes more blurry compared to the original.

In [ ]:
quant_1 = Quantization(threshold=2)(y_block_dct) 
quant_img_1 = Blocks().invert(Quantization().invert(quant_1)) 
quant_2 = Quantization(threshold=20)(y_block_dct) 
quant_img_2 = Blocks().invert(Quantization().invert(quant_2)) 

plt.subplot(331) 
plt.imshow(quant_img_1, cmap='gray', vmax = np.max(quant_img_1) * 0.01,vmin = 0) 
plt.subplot(332) 
plt.imshow(Blocks().invert(DCTofBlocks().invert(Quantization().invert(quant_1))), cmap='gray') 
plt.subplot(333) 
plt.imshow(y, cmap='gray') 
plt.subplot(334) 
plt.imshow(quant_img_2, cmap='gray', vmax = np.max(quant_img_2) * 0.01,vmin = 0) 
plt.subplot(335) 
plt.imshow(Blocks().invert(DCTofBlocks().invert(Quantization().invert(quant_2))), cmap='gray') 
plt.subplot(336) 
plt.imshow(y, cmap='gray') 
plt.show()

# Exercise 4 - Pick n-th highest - 2 Points
Implement the pick n-th highest algorithm

In [ ]:
class PickNthHighest:
    def __init__(self, n=1):
        self.n = n

    def __call__(self, blocks):
        """Pick the nth-highest frequencies"""
        # your code here
        zigzag = ZigZag(8) 
        zigzag_blocks = zigzag(blocks) 
        x,y = zigzag_blocks.shape[0], zigzag_blocks.shape[1]
        for i in range(x):
            for j in range(y):
                zigzag_blocks[i,j] = self.__pickHighest(zigzag_blocks[i,j])

        return zigzag.invert(zigzag_blocks)

    def __pickHighest(self, lis):
        result = np.zeros(len(lis))
        result[:self.n] = lis[:self.n]
        return result
        

    def invert(self, blocks):
        """There is no way to invert this operation. Just return the inputs."""
        return blocks

Try out pick n-th highest compression.

In [ ]:
pick_1 = PickNthHighest(n=16)(y_block_dct)
pick_img_1 = Blocks().invert(pick_1)
pick_2 = PickNthHighest(n=5)(y_block_dct)
pick_img_2 = Blocks().invert(pick_2)
plt.subplot(331)
plt.imshow(pick_img_1, cmap='gray', vmax = np.max(pick_img_1) * 0.01,vmin = 0)
plt.subplot(332)
plt.imshow(Blocks().invert(DCTofBlocks().invert(pick_1)), cmap='gray')
plt.subplot(333)
plt.imshow(y, cmap='gray')
plt.subplot(334)
plt.imshow(pick_img_2, cmap='gray', vmax = np.max(pick_img_2) * 0.01,vmin = 0)
plt.subplot(335)
plt.imshow(Blocks().invert(DCTofBlocks().invert(pick_2)), cmap='gray')
plt.subplot(336)
plt.imshow(y, cmap='gray')
plt.show()



In [ ]:
# build the jpeg pipeline
# for testing you can use only the first ones.
# maybe you have to adjust the Quantization threshold settings.
jpeg = Jpeg([Blocks(8), DCTofBlocks(), Quantization(threshold=10), ZigZag(8), Compress(np.int8)])

img_jpeg = jpeg(img)
img_reconstruct = jpeg.invert(img_jpeg)
assert img_reconstruct.shape == img.shape
# once you implemented ChromaSubsampling.invert this should have colors :)
plt.imshow(img_reconstruct / 255)
plt.show()

# Exercise 5 - Pipeline comparison - 2 Points
Compare different pipeline setups and complete the tasks below.

In [ ]:
# Compare the size of the images if the zigzag encoding is removed.
# Does the size change if the quantization threshold increases? 

thresholds = [2,5,10,20]
 
jpeg_size       = [ ]
jpeg_size_WZZ   = []
for i in thresholds:
    jpeg                = Jpeg([Blocks(8), DCTofBlocks(), Quantization(threshold=i), ZigZag(8), Compress(np.int8)])
    jpeg_withoutzigzag  = Jpeg([Blocks(8), DCTofBlocks(), Quantization(threshold=i), Compress(np.int8)])
    jpeg_img = jpeg(img)
    jpeg_img_WZZ = jpeg_withoutzigzag(img)
    jpeg_size.append(total_size_jpeg(jpeg_img))
    jpeg_size_WZZ.append(total_size_jpeg(jpeg_img_WZZ)) 




In [ ]:
labels = ['2','5','10','20']
plt.subplot(221)
plt.bar(list(range(len(labels))),jpeg_size)
plt.title('With ZIGZAG in KB')
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.xticks(list(range(len(thresholds))), labels)
plt.subplot(222)
plt.bar(list(range(len(labels))),jpeg_size_WZZ)
plt.title('without ZIGZAG in KB')
plt.xticks(list(range(len(labels))), labels)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.show()

In [ ]:
# Compare the image quality of the `Quantization` vs. the `PickNthHighest` compressions. Make sure that the outputs
# are roughly the same size. Why is one better then the other one?
# your code here

thresholds = [1,2,5,10] 
jpeg_size_quan   = []
jpeg_size_pickh  = []
for i in thresholds:
    jpeg_quan           = Jpeg([Blocks(8), DCTofBlocks(), Quantization(threshold=i), ZigZag(8), Compress(np.int8)])
    jpeg_pickh          = Jpeg([Blocks(8), DCTofBlocks(), PickNthHighest(n=i), ZigZag(8), Compress(np.int8)])
    jpeg_img_quan = jpeg_quan(img)
    jpeg_img_pickh = jpeg_pickh(img)


    jpeg_size_quan.append(total_size_jpeg(jpeg_img_quan))
    jpeg_size_pickh.append(total_size_jpeg(jpeg_img_pickh)) 

    plt.subplot(121)
    plt.title('quantization with threshold: ' + str(i))
    plt.imshow(jpeg_quan.invert(jpeg_img_quan))
    plt.subplot(122)
    plt.title('pick highest n=' +str(i))
    plt.imshow(jpeg_pickh.invert(jpeg_img_pickh))
    plt.show()

labels = [ '1','2','5','10']
plt.subplot(221)
plt.bar(list(range(len(labels))),jpeg_size_quan)
plt.title('quantization with threshold in KB')
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.xticks(list(range(len(labels))), labels)
plt.subplot(222)
plt.bar(list(range(len(labels))),jpeg_size_pickh)
plt.title('pick highest in KB')
plt.xticks(list(range(len(labels))), labels)
plt.grid(color='#95a5a6', linestyle='--', linewidth=2, axis='y', alpha=0.7)
plt.show()

# your short answer here
# With threshold=2 the images has almost the same size. Quantization with threshold 2 looks much better in compare to pick highest with n=2
# this effect is reached becuase with pick highest n=2 only the dc and the highest ac value are taken while with quantization threshold=2 
# all values are taken, which are > 2. Therefore prob. more AC values are choosen by using the quantization.   